In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

data_dir = Path("Data")
monthly_frames = []

# Load each month's data and store in a list
for month in range(1, 13):
    csv_path = data_dir / f"germany_2019_{month:02d}.csv"
    df = pd.read_csv(csv_path)  # first column becomes datetime
    monthly_frames.append(df)

# Combine all months into a single DataFrame with proper datetime parsing and sorting
frequency_data = pd.concat(monthly_frames, ignore_index=True)
frequency_data.rename(columns={"Unnamed: 0": "timestamp"}, inplace=True)
frequency_data['timestamp'] = pd.to_datetime(frequency_data['timestamp'], errors='coerce')
frequency_data = frequency_data.sort_values('timestamp').reset_index(drop=True)

In [2]:
# Scale frequency deviations to power per unit values 
min_abs, max_abs = 0, 200
mag_scaled = (frequency_data["Frequency"].abs() - min_abs) / (max_abs - min_abs)
mag_scaled = mag_scaled.clip(0, 1)
frequency_data["power_per_unit"] = mag_scaled * np.sign(frequency_data["Frequency"])

df = frequency_data.copy()


In [4]:
# Identify runs of consecutive positive or negative values -> aFRR would be activated at that point
sign = np.sign(df['power_per_unit']).fillna(0)

# Create a run identifier for consecutive segments of the same sign
run_id = (sign != sign.shift(fill_value=0)).cumsum()
df['_run_id'] = run_id

# Calculate elapsed time since the start of each run
run_start = df.groupby('_run_id')['timestamp'].transform('first')
elapsed = df['timestamp'] - run_start

# Mask for runs longer than 15 minutes with non-zero sign
mask_to_zero = (sign != 0) & (elapsed > pd.Timedelta(minutes=15))

# Create a new column for capped power per unit values with 15-minute rule applied
df['power_per_unit_capped'] = df['power_per_unit']
df.loc[mask_to_zero, 'power_per_unit_capped'] = 0.0

# Clean up temporary columns
df.drop(columns=['_run_id'], inplace=True, errors='ignore')

# Calculate energy in per unit hours -> Mwh per MW 
df['energy'] = df['power_per_unit_capped'] / 3600

# Resample to 4-hour intervals, summing the energy
energy_4h = (
    df
    .set_index('timestamp')
    .resample('4h')['energy']
    .sum()
    .reset_index()
)


In [ ]:
# Extract min, max, and mean energy values with timestamps
print(energy_4h.head())
min_row = energy_4h.loc[energy_4h["energy"].idxmin()]
max_row = energy_4h.loc[energy_4h["energy"].idxmax()]
mean_energy = energy_4h["energy"].mean()

p99_energy = energy_4h["energy"].abs().quantile(0.99)
p95_energy = energy_4h["energy"].abs().quantile(0.95)
p5_energy = energy_4h["energy"].abs().quantile(0.05)


print("Min:", min_row["energy"], "at", min_row["timestamp"])
print("Max:", max_row["energy"], "at", max_row["timestamp"])
print("Mean energy:", mean_energy)
print("99th percentile:", p99_energy)
print("95th percentile:", p95_energy)
print("5th percentile:", p5_energy)

energy_pos = energy_4h.loc[energy_4h["energy"] > 0, "energy"]
energy_neg = energy_4h.loc[energy_4h["energy"] < 0, "energy"]

p95_pos = energy_pos.quantile(0.95)      # 95th percentile of positives
p5_pos = energy_pos.quantile(0.05)       # 5th percentile of positives
p95_neg = energy_neg.abs().quantile(0.95)  # 95th percentile of negatives by magnitude
p5_neg = energy_neg.abs().quantile(0.05)   # 5th percentile of negatives by magnitude

print("95th percentile (only pos):", p95_pos)
print("5th percentile (only pos):", p5_pos)
print("95th percentile (only neg):", p95_neg)
print("5th percentile (only neg):", p5_neg)



            timestamp    energy
0 2019-01-01 00:00:00 -0.053147
1 2019-01-01 04:00:00  0.105717
2 2019-01-01 08:00:00  0.024122
3 2019-01-01 12:00:00 -0.028608
4 2019-01-01 16:00:00  0.020721
Min: -0.36657222222222224 at 2019-01-10 20:00:00
Max: 0.3036402777777778 at 2019-11-20 08:00:00
Mean energy: -0.004866448807711821
99th percentile: 0.2615803611111112
95th percentile: 0.2037033333333332
5th percentile: 0.0
95th percentile (only pos): 0.19581319444444445
5th percentile (only pos): 0.005578819444444444
95th percentile (only neg): 0.21832777777777776
5th percentile (only neg): 0.005583333333333334
